In [1]:
pip install nltk==3.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434677 sha256=a5cb1d23fd94e803dd48f4eb9e581ef86dc753e8fc0740bf20df699de1cb1f7b
  Stored in directory: /root/.cache/pip/wheels/35/ab/82/f9667f6f884d272670a15382599a9c753a1dfdc83f7412e37d
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1


In [2]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re 
import pandas as pd
import time
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import load_model

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
symptoms = pd.read_csv('/content/drive/MyDrive/CapstoneProjectBangkitMachineLearning/Data/dataset.csv')
symptoms.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
severity = pd.read_csv('/content/drive/MyDrive/CapstoneProjectBangkitMachineLearning/Data/Symptom-severity.csv')
severity.head()

In [ ]:
description = pd.read_csv('/content/drive/MyDrive/CapstoneProjectBangkitMachineLearning/Data/symptom_Description.csv')
description.head()

In [ ]:
precaution = pd.read_csv('/content/drive/MyDrive/CapstoneProjectBangkitMachineLearning/Data/symptom_precaution.csv')
precaution.head()

In [18]:
# Mengganti spaces dengan "_" dan menghapus spaces di bagian kiri kanan string
def substitute_spaces(string):
  if isinstance(string, str):
    string = string.strip()
    #string = re.sub(" ","_",string)
  return string

In [19]:
for cols in symptoms.columns:
  symptoms[cols] = symptoms[cols].apply(substitute_spaces)
for cols in severity.columns:
  severity[cols] = severity[cols].apply(substitute_spaces)

In [20]:
# Matching with symptoms data in severity
def matched_data(values):
  values = severity.loc[severity['Symptom'] == values]['weight']
  if len(values) > 0:
    return values.iloc[0]  # Return the first matching value
  else:
    return 0

In [21]:
cols = ['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4', 'Symptom_5', 
        'Symptom_6', 'Symptom_7', 'Symptom_8', 'Symptom_9', 'Symptom_10', 
        'Symptom_11', 'Symptom_12', 'Symptom_13', 'Symptom_14', 'Symptom_15', 
        'Symptom_16', 'Symptom_17']

for i in cols:
  symptoms[i] = symptoms[i].apply(matched_data)

In [22]:
symptoms.loc[symptoms['Disease'] == 'Dimorphic hemmorhoids(piles)', 'Disease'] = 'Dimorphic hemorrhoids(piles)'

In [23]:
# Matching with symptoms data in description
def label_encoding(values):
  for i in description.index:
    if values == description['Disease'].iloc[i]:
      values = i
  return values

In [24]:
symptoms['Disease'] = symptoms['Disease'].apply(label_encoding)

In [25]:
X = symptoms.drop(['Disease'], axis = 1)
y = symptoms['Disease']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(3936, 17) (3936,) (984, 17) (984,)


In [27]:
tf.random.set_seed(123)
num_labels = 41

# Define your model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_labels, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [28]:
# Train the model

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    verbose=1,
)

history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, batch_size=32)
print(f"Validation loss: {loss}")
print(f"Validation accuracy: {accuracy}")

Epoch 1/200
123/123 [==============================] - 1s 3ms/step - loss: 2.6143 - accuracy: 0.3410 - val_loss: 1.7369 - val_accuracy: 0.5539
Epoch 2/200
123/123 [==============================] - 0s 2ms/step - loss: 1.2931 - accuracy: 0.6700 - val_loss: 1.0446 - val_accuracy: 0.7124
Epoch 3/200
123/123 [==============================] - 0s 2ms/step - loss: 0.8359 - accuracy: 0.7896 - val_loss: 0.7405 - val_accuracy: 0.8242
Epoch 4/200
123/123 [==============================] - 0s 2ms/step - loss: 0.6248 - accuracy: 0.8498 - val_loss: 0.5972 - val_accuracy: 0.8618
Epoch 5/200
123/123 [==============================] - 0s 2ms/step - loss: 0.5089 - accuracy: 0.8763 - val_loss: 0.5012 - val_accuracy: 0.8831
Epoch 6/200
123/123 [==============================] - 0s 2ms/step - loss: 0.4306 - accuracy: 0.8974 - val_loss: 0.4074 - val_accuracy: 0.9177
Epoch 7/200
123/123 [==============================] - 0s 2ms/step - loss: 0.3696 - accuracy: 0.9096 - val_loss: 0.3608 - val_accuracy: 0.9024

In [ ]:
model.fit(X, y, epochs=200, batch_size=32, callbacks=[early_stopping])

In [30]:
# Menyimpan model ke dalam file
model.save('my_model')

In [31]:
import tensorflow as tf

# Memuat model dari file
model = tf.keras.models.load_model('my_model')

### input data

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load and preprocess training data
training_data = [...]  # Your training data
texts = [data['text'] for data in training_data]  # Extract text from training data

# Create and fit tokenizer on training data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

# Preprocess user input
user_input = 'This is an example sentence.'

# Convert input text into a sequence of word indices
input_sequence = tokenizer.texts_to_sequences([user_input])

# Pad the sequence to a fixed length (if needed)
max_length = 100  # Maximum sequence length
padded_sequence = pad_sequences(input_sequence, maxlen=max_length)

# Load and use the trained model
model = tf.keras.models.load_model('my_model.h5')

# Perform prediction on the preprocessed user input
prediction = model.predict(padded_sequence)


In [6]:
file_path = '/content/drive/MyDrive/CapstoneProjectBangkitMachineLearning/Data/dataset.csv'

In [7]:
import csv

# Fungsi untuk membaca data pelatihan dari file CSV
def load_training_data(file_path):
    training_data = {}
    with open(file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            disease = row['Disease']
            symptoms = [row[f'Symptom_{i}'] for i in range(1, 18)]
            training_data[disease] = symptoms
    return training_data

# Memuat data pelatihan dari file CSV
training_data = load_training_data('/content/drive/MyDrive/CapstoneProjectBangkitMachineLearning/Data/dataset.csv')

In [ ]:
# Function to show the training data
def show_training_data(data):
    for disease, symptoms in data.items():
        print(f"Disease: {disease}")
        print("Symptoms:", ", ".join(symptoms))
        print()

# Call the function to show the training data
show_training_data(training_data)

In [11]:
with open('/content/drive/MyDrive/CapstoneProjectBangkitMachineLearning/Coba Chatbot/chatbot.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


In [12]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [ ]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [ ]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#training model
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

In [ ]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
pip install colorama==0.4.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("/content/drive/MyDrive/CapstoneProjectBangkitMachineLearning/Coba_Chatbot/chatbot.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: hi
1/1 [==============================] - 0s 108ms/step
ChatBot: Hi
User: whats your name
1/1 [==============================] - 0s 35ms/step
ChatBot: I'm Joana!
User: thank you
1/1 [==============================] - 0s 24ms/step
ChatBot: My pleasure
User: stop
1/1 [==============================] - 0s 23ms/step
ChatBot: Hi
User: 

KeyboardInterrupt: ignored